# Надіслати завдання на обробку окремо вибраного відео обробнику video-Worker прямо шляхом публікації повідомленя в Redis

---
В даномій задачі Jupyter Notebooks  використовуються як інструменти тестування, відлагодження та моніторингу.

**На приклад:**
В уілях відладки ми можемо покласти якийсь сторонній файл у каталог **video** вказати його найменуваня в повідомленні для redis і подивитися результат роботи worker в логах:

```bash
docker compose logs video-worker -f

```



In [1]:
# Імпортуємо пакети
import os
import json
import time

import redis
from rq import Queue, Worker

# Читаю параметри підключення до Redis з env змінних
irds_host = os.getenv('RDS_HOST')
irds_port = os.getenv('RDS_PORT')
irds_psw = os.getenv('RDS_PSW')
irdsq_outmsg = os.getenv('RDSQ_OUTMSG')

# для debug
#print(f"Прочита значення змінних підключеня до Redis {irds_host} {irds_port} {irds_psw} {irdsq_outmsg}")


## Підключаюся до  Redis

In [2]:
# Саме підключення
red = redis.StrictRedis(irds_host, irds_port, password=irds_psw, decode_responses=False)

# Пробую  Ping
rping=red.ping()
print( f"Статус підключення: {rping} ")
q = None
q_msg = None

# Аналізуємо значення ping
if rping:
    print("До Redis підключидлися успішно")
    q = Queue(connection=red)
    q_msg = Queue( name=irdsq_outmsg, connection=red)
    print(f"Задач у черзі {irdsq_outmsg}: {len(q_msg)}")
else:
    print(" НE ПІДКЛЮЧИЛИСЯ ДО REDIS!!!")    

Статус підключення: True 
До Redis підключидлися успішно
Задач у черзі video_queue: 0


## Відправляємо повідмолення в Redis

In [3]:
#import json
print("Завантажую обробник черги....")
from utils import tasks
print("Завантаження виконано")
file_name="20260225-171507.avi"
try:
    # Публікуємо повідмолення в чергу для оборобки відое file_name
    message_o={"filename": file_name}
    message_s=json.dumps(message_o)
    print(f"Повідомлення для черги: {message_s}")
    job = q_msg.enqueue('utils.tasks.crttask_sendmsg',  message_s)
    print(f"Задачу додано в чергу: {job.id} для файлу {file_name}")
    # Список усіх ID завдань у черзі
    queued_job_ids = q_msg.job_ids
    print(f"ID завдань у черзі: {queued_job_ids}")

except Exception as e:
    print(f"Помилка черги Redis: {e}")

Завантажую обробник черги....
2026-02-25 17:39:24,220 - DEBUG - utils.tasks - debug message
2026-02-25 17:39:24,220 - DEBUG - utils.tasks - Модель завантажується з локального файлу: /usr/src/app/yolov8n.pt
2026-02-25 17:39:24,266 - DEBUG - utils.tasks - Модель успішно завантажено з локального файлу.
YOLOv8n summary: 129 layers, 3,157,200 parameters, 0 gradients, 8.9 GFLOPs
2026-02-25 17:39:24,340 - DEBUG - utils.tasks - YOLO model ready: (129, 3157200, 0, 8.8575488)
Завантаження виконано
Повідомлення для черги: {"filename": "20260225-171507.avi"}
Задачу додано в чергу: 0580f346-2a71-43ca-84ab-70ce3d0079ad для файлу 20260225-171507.avi
ID завдань у черзі: []


## Отримати статус тільки що відпавелного  завдання в чергу

In [4]:
import time
# 1. Отримуємо ID завдання (з попередньої клітинки)
current_job_id = job.id 
print(f"Задач у черзі {irdsq_outmsg}: {len(q_msg)}")
print(f"Checking status for Job ID: {current_job_id}")
# Кількість ітерацій
i=10
# 2. Цикл очікування результату
while i!=0:
    # Оновлюємо дані про завдання з Redis
    job.refresh()
    
    status = job.get_status()
    print(f"Current status: {status}")

    if status == 'finished':
        print("Done! Result:", job.result)
        break
    elif status == 'failed':
        print("Job failed! Check worker logs.")
        # Можна вивести помилку: print(job.exc_info)
        break
    i -= 1
    time.sleep(2) # Чекаємо 2 секунди перед наступною перевіркою

Задач у черзі video_queue: 0
Checking status for Job ID: 0580f346-2a71-43ca-84ab-70ce3d0079ad
Current status: JobStatus.STARTED
Current status: JobStatus.STARTED
Current status: JobStatus.FINISHED
Done! Result: {'ok': True, 'filename': '20260225-171507.avi', 'detected': ['person', 'vase'], 'count': 2, 'timestamp': '2026-02-25T17:39:35.362090'}


## Відобразити лог обробки повідомлень

In [5]:
import pandas as pd
import matplotlib.pyplot as plt

# Читаємо файл
df = pd.read_json('/app/video/detections_log.jsonl', lines=True)

# Перетворюємо час у нормальний формат
df['timestamp'] = pd.to_datetime(df['timestamp'])

display(df.head(50))

,ok,filename,detected,count,timestamp
0,True,20260223-144657.avi,[],0,2026-02-23 14:47:56.697779
1,True,20260223-144819.avi,"[person, tv, microwave, oven, keyboard]",5,2026-02-23 14:49:15.581433
2,True,20260223-144923.avi,[tv],1,2026-02-23 14:50:21.171055
3,True,20260223-144953.avi,"[person, bicycle, cow]",3,2026-02-23 14:50:53.121085
4,True,20260223-145148.avi,"[tv, person, cat]",3,2026-02-23 14:52:49.402444
5,True,20260223-145218.avi,"[cow, person, cat]",3,2026-02-23 14:53:18.309112
6,True,20260225-131639.avi,[traffic light],1,2026-02-25 13:17:26.873233
7,True,20260225-132752.avi,[traffic light],1,2026-02-25 13:28:39.076062
8,True,20260225-130240.avi,[],0,2026-02-25 16:47:42.777026
9,True,20260225-130240.avi,[],0,2026-02-25 16:52:27.875985
